In [1]:
# Load necessary packages
using CSV, JuMP, Gurobi, DataFrames

In [2]:
demand = CSV.read("Mock Demand.csv", header=true)
dist = CSV.read("Mock Time.csv", header=true)
dist = dist[:,2:size(dist,2)]
demand

,Client,A_i,B_i,D_i,Type
,Int64,Float64,Float64,Int64,String
1,1,15.0,17.0,1063,Client
2,2,19.0,21.0,1696,Client
3,3,7.0,25.0,12,Client
4,4,15.0,20.0,43,Client
5,5,20.0,25.0,178,Client
6,6,15.0,17.0,9,Client
7,7,12.55,23.9833,1,Client
8,8,15.8167,23.9833,1,Client
9,9,16.2,23.9833,1,Client


In [4]:
dist = convert(Matrix, dist);
demand = convert(Matrix, demand);

In [26]:
lab = 1; 
start_hub = 2;
end_hub = 4;
start_client = 5; 
end_client = 54; 

K = 15
target = sum(demand[:,4])
limit = 8

# Create model
m = Model(solver=GurobiSolver(OutputFlag = 0));

# Create variables
@variable(m, x[i=lab:end_client, j=lab:end_client, k=1:K], Bin); # All
@variable(m, e[j=lab:end_hub, k=1:K], Bin) # Hubs and lab
@variable(m, g[i=lab:end_client, k=1:K], Bin) # All
@variable(m, s[i=lab:end_client, k=1:K] >= 0) # All
@variable(m, q[i=start_hub:end_client, k=1:K] >= 0) # Clients and hubs
@variable(m, del[i=lab, k=1:K, h=19:24] >= 0) # Lab
@variable(m, inv[j=start_hub:end_hub, k=1:K] >= 0) # Hubs
@variable(m, y[k=1:K], Bin)
@variable(m, z[i=lab, k=1:K, h=19:24], Bin) # Lab

# Create objective
@objective(m, Min, sum(sum(sum((h-7)^2*del[i,k,h] for h in 19:24) for k in 1:K) for i in lab))

# Create constraints
@constraint(m, [i=lab:end_client, j=lab:end_client, k=1:K], x[i,j,k] <= y[k]) # i=all, j=all
@constraint(m, sum(y) <= K)
@constraint(m, [i=start_client:end_client], sum(sum(x[i,j,k] for k in 1:K) for j in lab:end_client) == 1) # i=clients, j=all
@constraint(m, [i=start_hub:end_hub], sum(sum(x[i,j,k] for k in 1:K) for j in lab:end_client) <= 1) # i=hubs, j=all
@constraint(m, [i=lab:end_client, j=lab:end_client, k=1:K], s[j,k] - s[i,k] <= limit) # i=all, j=all
@constraint(m, [k=1:K], sum(sum(x[i,j,k] for j in lab:end_client) for i in lab:end_client) + 
    sum(e[j,k] for j in lab:end_hub) == # j=all, i=all, j=hubs/lab
    sum(sum(x[j,i,k] for j in start_client:end_client) for i in lab:end_client) + 
    sum(g[i,k] for i in start_client:end_client)) # j=clients, i=all, i=clients
@constraint(m, [j=lab:end_hub, k=1:K], e[j,k] == sum(x[i,j,k] for i in start_client:end_client) - 
    sum(x[j,i,k] for i in lab:end_hub)) # j=hubs+lab, i=clients, i=hubs/lab
@constraint(m, [k=1:K], sum(g[:,k]) == 1)
@constraint(m, [k=1:K], sum(e[:,k]) == 1)
@constraint(m, [i=lab:end_hub, k=1:K], g[i,k] == 0) # i=hubs and lab
@constraint(m, [i=start_client:end_client, k=1:K], g[i,k] == sum(x[i,j,k] for j in lab:end_client) - 
    sum(x[j,i,k] for j in lab:end_client)) # i=clients, j=all
@constraint(m, [i=lab:end_hub, j=start_hub:end_client, k=1:K], x[i,j,k] == 0) # i=hubs+lab, j=clients+hubs
@constraint(m, [i=lab:end_client, k=1:K], x[i,i,k] == 0) # i=all
@constraint(m, [i=lab:end_client, j=lab:end_client, k=1:K], s[i,k] + dist[i,j] - 26*(1-x[i,j,k]) <= s[j,k]) # i=all, j=all
@constraint(m, [i=lab:end_client, k=1:K], demand[i,2] <= s[i,k]) # i=all
@constraint(m, [i=lab:end_client, k=1:K], demand[i,3] >= s[i,k]) # i=all
@constraint(m, [i=start_client:end_client], sum(q[i,:]) == demand[i,4]) # i=clients
@constraint(m, [i=start_client:end_client, k=1:K], q[i,k] <= 10000*sum(x[i,j,k] for j in lab:end_client)) # i=clients, j=all
@constraint(m, sum(sum(sum(del[i,k,h] for h in 19:24) for k=1:K) for i in lab) == target) # i=lab
@constraint(m, [i=lab, k=1:K, h=19:24], z[i,k,h] <= sum(x[j,i,k] for j in start_hub:end_client)) # i=lab, j=clients+hubs
@constraint(m, [k=1:K], sum(sum(z[i,k,h] for i in lab) for h in 19:24) <= 1) # i=lab
@constraint(m, [j=start_hub:end_hub, k=1:K], sum(q[i,k] for i in start_client:end_client) - 
    10000*(1-sum(x[i,j,k] for i in start_client:end_client)) <= inv[j,k]) # i=clients, j=hubs
@constraint(m, [j=start_hub:end_hub, k=1:K], sum(q[i,k] for i in start_client:end_client) + 
    10000*(1-sum(x[i,j,k] for i in start_client:end_client)) >= inv[j,k]) # j=hubs, i=clients
@constraint(m, [j=start_hub:end_hub, k=1:K], inv[j,k] <= 10000*sum(x[i,j,k] for i in start_hub:end_hub)) # j=hubs, i=clients
@constraint(m, [j=start_hub:end_hub], sum(q[j,k] for k in 1:K) == sum(inv[j,k] for k in 1:K)) # j=hubs
@constraint(m, [j=start_hub:end_hub, k=1:K], q[j,k] <= 10000*sum(x[j,i,k] for i in lab:end_hub)) # j=hubs, i=hubs+lab
@constraint(m, [i=lab, k=1:K, h=19:24], -10000*(1-z[i,k,h]) <= s[i,k] - h) # i=lab
@constraint(m, [i=lab, k=1:K, h=19:24], 1 + 10000*(1-z[i,k,h]) >= s[i,k] - h) # i=lab
@constraint(m, [j=lab, k=1:K, h=19:24], -10000*(1-z[j,k,h]) + 
    sum(q[i,k] for i in start_hub:end_client) <= del[j,k,h]) # j=lab, i=clients+hubs
@constraint(m, [j=lab, k=1:K, h=19:24], 10000*(1-z[j,k,h]) + 
    sum(q[i,k] for i in start_hub:end_client) >= del[j,k,h]) # j=lab, i=clients+hubs

solve(m)

Academic license - for non-commercial use only


:Optimal

In [24]:
(getvalue(s))

54×16 Array{Float64,2}:
 15.0     17.0     17.0     17.0     …  17.0     17.0     17.0     17.0   
 19.0     19.0     19.0     19.0        19.0     19.0     19.0     19.0   
 14.0     14.0     13.9833  14.0        14.0     14.0     14.0     14.0   
 15.0     15.0     15.0     15.0        15.0     15.0     15.0     15.0   
 20.0     20.0     20.0     20.0        20.0     20.0     20.0     20.0   
 15.0     15.0     17.0     15.0     …  15.0     15.0     15.0     15.0   
 14.0     14.0     13.9833  14.0        14.0     14.0     12.55    14.0   
 15.8167  15.8167  15.8167  15.8167     15.8167  15.8167  15.8167  15.8167
 16.2     16.2     16.2     19.85       16.2     16.2     16.2     16.2   
 16.0     14.0     13.9833  14.0        14.0     14.0     14.0     14.0   
 18.5     18.5     18.5     18.5     …  18.5     18.5     18.5     18.5   
 23.0     14.3     14.3     14.3        14.3     14.3     14.3     14.3   
 14.0     14.0     14.0     14.0        14.0     14.0     14.0     14.0   
 

In [25]:
findall(getvalue(x) .> 0)

50-element Array{CartesianIndex{3},1}:
 CartesianIndex(1, 10, 1)  
 CartesianIndex(38, 12, 1) 
 CartesianIndex(10, 14, 1) 
 CartesianIndex(35, 21, 1) 
 CartesianIndex(25, 22, 1) 
 CartesianIndex(26, 23, 1) 
 CartesianIndex(23, 25, 1) 
 CartesianIndex(21, 26, 1) 
 CartesianIndex(14, 35, 1) 
 CartesianIndex(22, 36, 1) 
 CartesianIndex(36, 38, 1) 
 CartesianIndex(12, 54, 1) 
 CartesianIndex(20, 54, 2) 
 ⋮                         
 CartesianIndex(18, 54, 9) 
 CartesianIndex(5, 15, 10) 
 CartesianIndex(15, 54, 10)
 CartesianIndex(45, 54, 11)
 CartesianIndex(47, 8, 12) 
 CartesianIndex(8, 54, 12) 
 CartesianIndex(31, 54, 13)
 CartesianIndex(46, 54, 14)
 CartesianIndex(7, 19, 15) 
 CartesianIndex(19, 34, 15)
 CartesianIndex(34, 54, 15)
 CartesianIndex(42, 54, 16)

In [22]:
getvalue(s[29,1])

19.669999999999806

In [10]:
getvalue(e)

e: 2 dimensions:
[51,:]
  [51, 1] = 0.0
  [51, 2] = 0.0
  [51, 3] = 0.0
  [51, 4] = 0.0
  [51, 5] = 0.0
  [51, 6] = 0.0
  [51, 7] = 0.0
  [51, 8] = 0.0
  [51, 9] = 0.0
  [51,10] = 0.0
  [51,11] = 0.0
  [51,12] = 0.0
  [51,13] = 0.0
  [51,14] = 0.0
  [51,15] = 0.0
  [51,16] = 0.0
  [51,17] = 0.0
  [51,18] = 0.0
  [51,19] = 0.0
  [51,20] = 0.0
[52,:]
  [52, 1] = 0.0
  [52, 2] = 0.0
  [52, 3] = 0.0
  [52, 4] = 0.0
  [52, 5] = 0.0
  [52, 6] = 0.0
  [52, 7] = 0.0
  [52, 8] = 0.0
  [52, 9] = 0.0
  [52,10] = 0.0
  [52,11] = 0.0
  [52,12] = 0.0
  [52,13] = 0.0
  [52,14] = 0.0
  [52,15] = 0.0
  [52,16] = 0.0
  [52,17] = 0.0
  [52,18] = 0.0
  [52,19] = 0.0
  [52,20] = 0.0
[53,:]
  [53, 1] = 0.0
  [53, 2] = 0.0
  [53, 3] = 0.0
  [53, 4] = 0.0
  [53, 5] = 0.0
  [53, 6] = 0.0
  [53, 7] = 0.0
  [53, 8] = 0.0
  [53, 9] = 0.0
  [53,10] = 0.0
  [53,11] = 0.0
  [53,12] = 0.0
  [53,13] = 0.0
  [53,14] = 0.0
  [53,15] = 0.0
  [53,16] = 0.0
  [53,17] = 0.0
  [53,18] = 0.0
  [53,19] = 0.0
  [53,20] = 0.0
[5